# Traffic Light Counting

## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 20.8 MB/s eta 0:00:00


In [2]:
import cv2
from ultralytics import YOLO, solutions
# from google.colab import drive
# drive.mount('/content/drive')

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [3]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 128MB/s]


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [4]:
cap = cv2.VideoCapture('/content/Video_MiniProject.MP4')
assert cap.isOpened(), 'The file dose not Found'

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [5]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [6]:
video_writer = cv2.VideoWriter("Counting_Line.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

## Process Video Frames and Identify Vehicles

In [7]:
color = (0, 245, 0)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7
font_thickness = 1

ret = True
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    # Initialize class counts
    class_counts = {}

    results = model.track(frame, persist=True)

    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()  # Bounding box coordinates
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1  # Class ID
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0  # Confidence score
            id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1  # Unique ID

            # Update class count
            if class_id not in class_counts:
                class_counts[class_id] = 0
            class_counts[class_id] += 1

            # Convert bbox coordinates to integers
            x1, y1, x2, y2 = map(int, bbox)

            # Draw the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            # Prepare the label
            label = f'Count: {id} Class: {class_id} Prob: {conf:.2f}'

            # Put the label on the frame
            cv2.putText(frame, label, (x1, y1 - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)

    # Draw class counts at the bottom of the frame
    text_offset = 20
    for class_id, count in class_counts.items():
        label = f'Class {class_id}: {count}'
        cv2.putText(frame, label, (10, height - text_offset), font, font_scale, color, font_thickness, cv2.LINE_AA)
        text_offset += 20  # Move to the next line

    video_writer.write(frame)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
0: 320x640 1 car, 10.1ms
Speed: 2.1ms preprocess, 10.1ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 9.0ms
Speed: 2.5ms preprocess, 9.0ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 8.6ms
Speed: 2.1ms preprocess, 8.6ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 11.3ms
Speed: 2.1ms preprocess, 11.3ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 8.2ms
Speed: 2.2ms preprocess, 8.2ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 7.8ms
Speed: 2.1ms preprocess, 7.8ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 11.6ms
Speed: 2.2ms preprocess, 11.6ms inference, 1.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 14.4ms
Speed: 2.4ms preprocess, 14.4ms inference, 2.3ms postprocess per image